In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import re
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split as tts
import torch
import collections 
from collections import Counter
from torch import nn
from torch.utils.data import Dataset, DataLoader
from bs4 import BeautifulSoup 
import math

In [2]:
import nltk
nltk_data_directory = "/projectnb/ds598/projects/xthomas/misc/nltk_data"
nltk.data.path.append(nltk_data_directory)
nltk.download('stopwords', download_dir=nltk_data_directory)

[nltk_data] Downloading package stopwords to
[nltk_data]     /projectnb/ds598/projects/xthomas/misc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Loading and Cleaning the Data

In [3]:
df = pd.read_csv('/projectnb/ds598/admin/xthomas/sp2024_notebooks/discussion/disc5/data/Reviews.csv', nrows = 10000)
data = df[['Text', 'Summary']]
data.drop_duplicates(subset=['Text'], inplace=True) # Drop duplicate rows

In [4]:
data.head()

,Text,Summary
0,I have bought several of the Vitality canned d...,Good Quality Dog Food
1,Product arrived labeled as Jumbo Salted Peanut...,Not as Advertised
2,This is a confection that has been around a fe...,"""Delight"" says it all"
3,If you are looking for the secret ingredient i...,Cough Medicine
4,Great taffy at a great price. There was a wid...,Great taffy


In [5]:
word_mapping = {"ain't": "is not","aint": "is not", "aren't": "are not","arent": "are not","can't": "cannot","cant": "cannot", "'cause": "because", "cause": "because", "could've": "could have", "couldn't": "could not",

     "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

     "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

     "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

     "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

     "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

     "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", 'mstake':"mistake",

     "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

     "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

     "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

     "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

     "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

     "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

     "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

     "wasn't": "was not",'wasnt':"was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

     "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

     "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

     "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

     "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

     "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

     "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

     "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

     "you're": "you are", "you've": "you have", 'youve':"you have", 'goin':"going", '4ward':"forward", "shant":"shall not",'tat':"that", 'u':"you", 'v': "we",'b4':'before', "sayin'":"saying"
}

In [6]:
stop_words = set(stopwords.words('english'))
def text_cleaner(text):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([word_mapping[t] if t in word_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = [w for w in newString.split() if not w in stop_words]
    long_words=[]
    
    tokens = [w for w in newString.split() if not w in stop_words]
    long_words=[]
    for i in tokens:
        if len(i)>=3:                  #removing short word
            long_words.append(i) 
    text = " ".join(long_words).strip()
    def no_space(word, prev_word):
        return word in set(',!"";.''?') and prev_word!=" "
    text = text.replace('\u202f', ' ').replace('\xa0', ' ').lower()
    out = [' ' + char if i > 0 and no_space(char, text[i - 1]) else char for i, char in enumerate(text)]
    text = ''.join(out)
    return text

data['cleaned_text'] = data['Text'].apply(text_cleaner)
data['cleaned_summary'] = data['Summary'].apply(text_cleaner)
# this step is to remove all rows that have a blank summary
data["cleaned_summary"].replace('', np.nan, inplace=True)
data.dropna(axis=0, inplace=True)

In [7]:
max_len_text=100 
max_len_summary=10

In [8]:
x_train,x_test,y_train,y_test = tts(data['cleaned_text'],data['cleaned_summary'],test_size=0.1, shuffle=True, random_state=111)

In [9]:
# Tokenize function 
def tokenize(lines, token='word'):
    assert token in ('word', 'char'), 'Unknown token type: ' + token
    return [line.split() if token == 'word' else list(line) for line in lines]

# pading function
def truncate_pad(line, num_steps, padding_token):
    if len(line) > num_steps:
        return line[:num_steps]  # Truncate
    return line + [padding_token] * (num_steps - len(line))  # Pad

# the vocabulary class 
class Vocab:
    def __init__(self, tokens=[], min_freq=0, reserved_tokens=[]):
        # Flatten a 2D list if needed
        if tokens and isinstance(tokens[0], list):
            tokens = [token for line in tokens for token in line]
        # Count token frequencies
        counter = collections.Counter(tokens)
        self.token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                  reverse=True)
        # The list of unique tokens
        self.idx_to_token = list(sorted(set(['<unk>'] + reserved_tokens + [
            token for token, freq in self.token_freqs if freq >= min_freq])))
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if hasattr(indices, '__len__') and len(indices) > 1:
            return [self.idx_to_token[int(index)] for index in indices]
        return self.idx_to_token[indices]

    def unk(self):  # Index for the unknown token
        return self.token_to_idx['<unk>']
# tokenize
src_tokens = tokenize(x_train)
tgt_tokens = tokenize(y_train)
# build vocabulary on dataset
src_vocab = Vocab(src_tokens, reserved_tokens=['<pad>', '<bos>', '<eos>'])
tgt_vocab = Vocab(tgt_tokens, reserved_tokens=['<pad>', '<bos>', '<eos>'])

In [10]:
# fn to add eos and padding and also determine valid length of each data sample
def build_array_sum(lines, vocab, num_steps):
    lines = [vocab[l] for l in lines]
    lines = [l + [vocab['<eos>']] for l in lines]
    array = torch.tensor([truncate_pad(l, num_steps, vocab['<pad>']) for l in lines])
    valid_len = (array != vocab['<pad>']).type(torch.int32).sum(1)
    return array, valid_len

src_array, src_valid_len = build_array_sum(src_tokens, src_vocab, max_len_text)
tgt_array, tgt_valid_len = build_array_sum(tgt_tokens, tgt_vocab, max_len_summary)
data_arrays = (src_array, src_valid_len, tgt_array, tgt_valid_len)

In [11]:
# create the tensor dataset object 
def load_array(data_arrays, batch_size, is_train=True):
    dataset = torch.utils.data.TensorDataset(*data_arrays)
    return torch.utils.data.DataLoader(dataset, batch_size, shuffle=is_train)
batch_size = 64
data_iter = load_array(data_arrays, batch_size)

### Example

Suppose we have an input sequence of four tokens: “The cat sat on the mat”. We want to compute the self-attention output for the token “cat”. We first embed each token into a vector of size dq, and then we form the query, key, and value matrices by stacking the vectors vertically. For simplicity, we assume that the query, key, and value matrices are the same, and we use the following values for the vectors:

In [33]:
# The input sequence
tokens = ["The", "cat", "sat", "on", "the", "mat"]

# The embedding vectors
vectors = [
    [0.1, 0.2, 0.3, 0.4], # The
    [0.5, 0.6, 0.7, 0.8], # cat
    [0.9, 1.0, 1.1, 1.2], # sat
    [1.3, 1.4, 1.5, 1.6], # on
    [1.7, 1.8, 1.9, 2.0], # the
    [2.1, 2.2, 2.3, 2.4]  # mat
]

# The query, key, and value matrices
Q = K = V = [
    [0.1, 0.2, 0.3, 0.4], # The
    [0.5, 0.6, 0.7, 0.8], # cat
    [0.9, 1.0, 1.1, 1.2], # sat
    [1.3, 1.4, 1.5, 1.6], # on
    [1.7, 1.8, 1.9, 2.0], # the
    [2.1, 2.2, 2.3, 2.4]  # mat
]

# Q, K, V initialzed the same for simplicity

To compute the self-attention output for the token “cat”, we use the second row of the query matrix Q as the query vector, and we compute the dot product of the query vector with each row of the key matrix K. This gives us the following attention scores:

In [32]:
# The query vector for "cat"
q = [0.5, 0.6, 0.7, 0.8]

# The dot product of the query vector with each row of the key matrix
scores = [
    0.5 * 0.1 + 0.6 * 0.2 + 0.7 * 0.3 + 0.8 * 0.4, # The
    0.5 * 0.5 + 0.6 * 0.6 + 0.7 * 0.7 + 0.8 * 0.8, # cat
    0.5 * 0.9 + 0.6 * 1.0 + 0.7 * 1.1 + 0.8 * 1.2, # sat
    0.5 * 1.3 + 0.6 * 1.4 + 0.7 * 1.5 + 0.8 * 1.6, # on
    0.5 * 1.7 + 0.6 * 1.8 + 0.7 * 1.9 + 0.8 * 2.0, # the
    0.5 * 2.1 + 0.6 * 2.2 + 0.7 * 2.3 + 0.8 * 2.4  # mat
]

# The attention scores
scores = [0.9, 2.2, 3.5, 4.8, 6.1, 7.4]

Next, we divide the attention scores by the square root of the dimension of the query, which is 4 in this case. This gives us the following scaled attention scores:

In [36]:
# The square root of the dimension of the query vector
sqrt_dq = 2

# The scaled attention scores
scaled_scores = [0.45, 1.1, 1.75, 2.4, 3.05, 3.7]

Then, we apply the softmax function to the scaled attention scores to obtain the attention weights. The softmax function normalizes the scores to a probability distribution, where the sum of the weights is 1 and the higher the score, the higher the weight. This gives us the following attention weights:

In [37]:
# The softmax function
def softmax(x):
    # Compute the exponential of each element
    exp_x = [math.exp(e) for e in x]
    # Compute the sum of the exponential elements
    sum_exp_x = sum(exp_x)
    # Divide each element by the sum
    return [e / sum_exp_x for e in exp_x]

# The attention weights
weights = softmax(scaled_scores)

# The attention weights
weights = [0.001, 0.004, 0.016, 0.054, 0.182, 0.743]

We then multiply the attention weights by the value matrix V to obtain the output. The output is a weighted sum of the value vectors, where the weight of each vector is determined by the relevance or similarity of the corresponding token to the query token. This gives us the following output:

In [38]:
# The output
output = [
    weights[0] * V[0][0] + weights[1] * V[1][0] + weights[2] * V[2][0] + weights[3] * V[3][0] + weights[4] * V[4][0] + weights[5] * V[5][0], # First element
    weights[0] * V[0][1] + weights[1] * V[1][1] + weights[2] * V[2][1] + weights[3] * V[3][1] + weights[4] * V[4][1] + weights[5] * V[5][1], # Second element
    weights[0] * V[0][2] + weights[1] * V[1][2] + weights[2] * V[2][2] + weights[3] * V[3][2] + weights[4] * V[4][2] + weights[5] * V[5][2], # Third element
    weights[0] * V[0][3] + weights[1] * V[1][3] + weights[2] * V[2][3] + weights[3] * V[3][3] + weights[4] * V[4][3] + weights[5] * V[5][3]  # Fourth element
]

# The output
output = [1.58, 1.68, 1.78, 1.88]

## Coding The Transformer

![title](https://miro.medium.com/v2/resize:fit:888/format:webp/1*uRF3gFWpXbtl6pnFe5mOlg.jpeg)

Two main parts to a transformer:

- Encoder:  The encoder takes in the input sentence and produces a fixed-size vector representation of it
- Decoder: The decoder takes the fixed-size vector representation and produces the output sentence. The decoder uses both self-attention and cross-attention, where the attention mechanism is applied to the output of the encoder and the input of the decoder.

#### Muli-head Attention

This multi-head attention mechanism allows the model to capture various aspects of the input data by processing it in parallel through multiple attention heads. The use of dot-product attention within each head enables the model to dynamically focus on different parts of the input sequence, enhancing its ability to model complex dependencies.

In [12]:
# The main class 
class MultiHeadAttention(nn.Module):
    def __init__(self, key_size, query_size, value_size, num_hiddens, num_heads, dropout, bias=False, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = DotProductAttention(dropout)
        self.w_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.w_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.w_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.w_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)
    def forward(self, queries, keys, values, valid_lens):
        queries = transpose_qkv(self.w_q(queries), self.num_heads)
        keys = transpose_qkv(self.w_k(keys), self.num_heads)
        values = transpose_qkv(self.w_v(values), self.num_heads)
        if valid_lens is not None:
            valid_lens = torch.repeat_interleave(valid_lens, repeats = self.num_heads, dim=0)
        output = self.attention(queries, keys, values, valid_lens)
        output_concat = transpose_output(output, self.num_heads)
        return self.w_o(output_concat)

# Function to transpose the linearly transformed query key and values 
def transpose_qkv(X, num_heads):
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)
    X = X.permute(0, 2, 1, 3)
    return X.reshape(-1, X.shape[2], X.shape[3])

# For output formatting 
def transpose_output(X, num_heads):
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)

# The dot product attention scoring function 
class DotProductAttention(nn.Module):
    def __init__(self, dropout, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
    def forward(self, queries, keys, values, valid_lens=None):
        d = queries.shape[-1]
        scores = torch.bmm(queries, keys.transpose(1, 2))/math.sqrt(d)
        self.attention_weights = masked_softmax(scores, valid_lens)
        return torch.bmm(self.dropout(self.attention_weights), values)
# Here masking is used so that irrelevant padding tokens are not considered
# while calculations

def sequence_mask(X, valid_len, value=0):
    maxlen = X.size(1)
    mask = torch.arange((maxlen), dtype=torch.float32)[None, :] < valid_len[:, None]    #device=X.device
    X[~mask] = value
    return X
# the irrelevant tokens are given a very small negative value which gets
# ignored in the subsequent calculations
def masked_softmax(X, valid_lens):
    if valid_lens is None:
        return nn.functional.softmax(X, dim=-1)
    else:
        shape = X.shape
        if valid_lens.dim() == 1:
            valid_lens = torch.repeat_interleave(valid_lens, shape[1])
        else:
            valid_lens = valid_lens.reshape(-1) 
        X = sequence_mask(X.reshape(-1, shape[-1]), valid_lens, value=-1e6)
        return nn.functional.softmax(X.reshape(shape), dim=-1)

`MultiHeadAttention` class:
transforms the input queries, keys, and values using the initialized weight matrices, splits these transformations into multiple heads, applies dot-product attention in each head, concatenates the heads' outputs, and finally linearly transforms the concatenated output.    

`DotProductAttention` class:
computes the attention scores by performing a dot product between queries and keys, scales these scores by the square root of the query dimension, applies a mask based on valid lengths to ignore padding, applies softmax to get probabilities, and finally computes the weighted sum of values.

`Masking` Functions:
- sequence_mask creates a mask for the input tensor based on valid lengths, setting values to 0 (or another specified value) for positions beyond each sequence's valid length to ignore them in subsequent calculations.    
- masked_softmax applies softmax to only the valid parts of each sequence, ignoring masked parts by setting their values to a large negative number before softmax, effectively making their contribution to the softmax output negligible.    


In [13]:
class PositionWiseFFN(nn.Module):
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_output, **kwargs):
        super(PositionWiseFFN, self).__init__(**kwargs)
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_output)
    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))

`PositionWiseFFN` class:
- Transforms the input tensor with two dense layers and an activation function (ReLU) in between. This is applied to each position separately and identically.

In [14]:
class PositionalEncoding(nn.Module):
    def __init__(self, num_hiddens, dropout, max_len=1000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.P = torch.zeros((1, max_len, num_hiddens))
        X = torch.arange(max_len, dtype=torch.float32).reshape(-1, 1)/torch.pow(10000,torch.arange(0, num_hiddens,2, dtype=torch.float32)/num_hiddens)
        self.P[:,:, 0::2] = torch.sin(X)
        self.P[:, :, 1::2] = torch.cos(X)
    
    def forward(self, X):
        X = X + self.P[:, :X.shape[1], :].to(X.device)
        return self.dropout(X)


`PositionalEncoding` class:
- Adds positional information to the input embeddings by summing them with positional encodings of the same dimensions. The positional encodings are calculated using sine and cosine functions of different frequencies.

In [15]:
class AddNorm(nn.Module):
    def __init__(self, normalized_shape, dropout, **kwargs):
        super(AddNorm, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)
    def forward(self, X, Y):
        return self.ln(self.dropout(Y) + X)

In [16]:
# class for the block structure within 
class EncoderBlock(nn.Module):
    def __init__(self, key_size, query_size, value_size, num_hiddens, norm_shape, ffn_num_input, 
                 ffn_num_hiddens, num_heads, dropout, use_bias=False, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)
        self.attention = MultiHeadAttention(key_size, query_size, value_size, num_hiddens,num_heads, dropout, use_bias)
        self.addnorm1 = AddNorm(norm_shape, dropout)
        self.ffn = PositionWiseFFN(ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm2 = AddNorm(norm_shape, dropout)
        
    def forward(self, X, valid_lens):
        Y = self.addnorm1(X, self.attention(X, X, X, valid_lens))
        return self.addnorm2(Y, self.ffn(Y))

# the main encoder class
class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, key_size, query_size, value_size, num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens, num_heads, num_layers, dropout, use_bias=False, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(vocab_size, num_hiddens)
        self.pos_encoding = PositionalEncoding(num_hiddens, dropout)
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module("block"+str(i),EncoderBlock(key_size, query_size, value_size, num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens, num_heads, dropout, use_bias))
    
    def forward(self, X, valid_lens, *args):
        X = self.pos_encoding(self.embedding(X)*math.sqrt(self.num_hiddens))
        self.attention_weights = [None]*len(self.blks)
        for i, blk in enumerate(self.blks):
            X = blk(X, valid_lens)
            self.attention_weights[i] = blk.attention.attention.attention_weights
        return X

![title](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*9Tk36lXDqBxXjgn0Fne60g.jpeg)

In [17]:
class DecoderBlock(nn.Module):
    def __init__(self, key_size, query_size, value_size, num_hiddens, norm_shape,
                 ffn_num_input, ffn_num_hiddens, num_heads, dropout, i, **kwargs):
        super(DecoderBlock, self).__init__(**kwargs)
        self.i = i
        self.attention1 = MultiHeadAttention(key_size, query_size, value_size, num_hiddens, num_heads, dropout)
        self.addnorm1 = AddNorm(norm_shape, dropout)
        self.attention2 = MultiHeadAttention(key_size, query_size, value_size, num_hiddens, num_heads, dropout)
        self.addnorm2 = AddNorm(norm_shape, dropout)
        self.ffn = PositionWiseFFN(ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm3 = AddNorm(norm_shape, dropout)
        
    def forward(self, X, state):
        enc_outputs, enc_valid_lens = state[0], state[1]
        if state[2][self.i] is None: # true when training the model
            key_values = X
        else:                        # while decoding state[2][self.i] is decoded output of the ith block till the present time-step
            key_values = torch.cat((state[2][self.i], X), axis=1)
        state[2][self.i] = key_values
        if self.training:
            batch_size, num_steps, _ = X.shape
            dec_valid_lens = torch.arange(1, num_steps+1, device = X.device).repeat(batch_size, 1)
        else:
            dec_valid_lens = None
        X2 = self.attention1(X, key_values, key_values, dec_valid_lens)
        Y = self.addnorm1(X, X2)
        Y2 = self.attention2(Y, enc_outputs, enc_outputs, enc_valid_lens)
        Z = self.addnorm2(Y, Y2)
        return self.addnorm3(Z, self.ffn(Z)), state

# The main decoder class 
class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size, key_size, query_size, value_size, num_hiddens,
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads, num_layers, dropout, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.num_layers = num_layers
        self.embedding = nn.Embedding(vocab_size, num_hiddens)
        self.pos_encoding = PositionalEncoding(num_hiddens, dropout)
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module("block"+str(i), 
                                DecoderBlock(key_size, query_size, value_size,
                                             num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens, num_heads, dropout, i))
            self.dense = nn.Linear(num_hiddens, vocab_size)
    
    def init_state(self, enc_outputs, enc_valid_lens, *args):
        return [enc_outputs, enc_valid_lens, [None]*self.num_layers]
    
    def forward(self, X, state):
        X = self.pos_encoding(self.embedding(X)*math.sqrt(self.num_hiddens))
        self._attention_weights = [[None]*len(self.blks) for _ in range(2)]
        for i, blk in enumerate(self.blks):
            X, state = blk(X, state)
            self._attention_weights[0][i] = blk.attention1.attention.attention_weights
            self._attention_weights[1][i] = blk.attention2.attention.attention_weights
        return self.dense(X), state
    
    def attention_weights(self):
        return self._attention_weights

In [18]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, enc_X, dec_X, *args):
        enc_all_outputs = self.encoder(enc_X, *args)
        dec_state = self.decoder.init_state(enc_all_outputs, *args)
        # Return decoder output only
        return self.decoder(dec_X, dec_state)[0]

In [40]:
# Using built-in transformer module

transformer_ = nn.Transformer(d_model=512, nhead=2, num_encoder_layers=6, num_decoder_layers=6, dim_feedforward=2048, dropout=0.1)
print(transformer_)

Transformer(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
    (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (decoder): TransformerDecoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, o

In [19]:
def get_device(i=0):
    if torch.cuda.device_count() >= i+1:
        return torch.device(f'cuda:{i}')
    else:
        return torch.device('cpu')
device = get_device()

In [20]:
num_hiddens, num_layers, dropout, num_steps = 32, 2, 0.1, 10
ffn_num_input, ffn_num_hiddens, num_heads = 32, 64, 4
key_size, query_size, value_size = 32, 32, 32
norm_shape = [32]
encoder = TransformerEncoder(len(src_vocab), key_size, query_size, value_size, num_hiddens,norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,num_layers, dropout)
decoder = TransformerDecoder(len(tgt_vocab), key_size, query_size, value_size, num_hiddens,norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,num_layers, dropout)
net = Transformer(encoder, decoder) # for initialising the weights of the fully connected layers in the model

In [21]:
def grad_clipping(net, theta):
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [41]:
# built in gradient clipping to avoid exploding gradients during training.
# torch.utils.clip_grad_norm_(net.parameters(), 1)


In [22]:
class Accumulator:
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [23]:
class MaskedSoftmaxCELoss(nn.CrossEntropyLoss):
    # `pred` shape: (`batch_size`, `num_steps`, `vocab_size`)
    # `label` shape: (`batch_size`, `num_steps`)
    # `valid_len` shape: (`batch_size`,)
    def forward(self, pred, label, valid_len):
        weights = torch.ones_like(label)
        weights = sequence_mask(weights, valid_len)
        self.reduction='none'
        unweighted_loss = super(MaskedSoftmaxCELoss, self).forward(pred.permute(0, 2, 1), label)
        weighted_loss = (unweighted_loss * weights).mean(dim=1)
        return weighted_loss

In [24]:
def train_seq2seq(net, data_iter, lr, num_epochs, tgt_vocab, device):    
    net.to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    loss = MaskedSoftmaxCELoss()
    net.train()
    for epoch in range(num_epochs):
        metric = Accumulator(2)  # Sum of training loss, no. of tokens
        for batch in data_iter:
            optimizer.zero_grad()
            X, X_valid_len, Y, Y_valid_len = [x.to(device) for x in batch]
            bos = torch.tensor([tgt_vocab['<bos>']] * Y.shape[0],device=device).reshape(-1, 1)
            dec_input = torch.cat([bos, Y[:, :-1]], 1)  # Teacher forcing
            Y_hat = net(X, dec_input, X_valid_len)
            l = loss(Y_hat, Y, Y_valid_len)
            l.sum().backward()  # Make the loss scalar for `backward`
            grad_clipping(net, 1)
            num_tokens = Y_valid_len.sum()
            optimizer.step()
            with torch.no_grad():
                metric.add(l.sum(), num_tokens)
        print(f"Done with epoch number: {epoch+1}") # optional step
    print(f'loss {metric[0] / metric[1]:.3f} on {str(device)}')

In [25]:
lr = 0.005
num_epochs = 10
train_seq2seq(net, data_iter, lr, num_epochs, tgt_vocab, device)

Done with epoch number: 1
Done with epoch number: 2
Done with epoch number: 3
Done with epoch number: 4
Done with epoch number: 5
Done with epoch number: 6
Done with epoch number: 7
Done with epoch number: 8
Done with epoch number: 9
Done with epoch number: 10
loss 0.389 on cpu


In [26]:
def predict_seq2seq(net, src_sentence, src_vocab, tgt_vocab, num_steps,device, save_attention_weights=False):
    # Set `net` to eval mode for inference
    net.eval()
    src_tokens = src_vocab[src_sentence.lower().split(' ')] + [src_vocab['<eos>']]
    enc_valid_len = torch.tensor([len(src_tokens)], device=device)
    src_tokens = truncate_pad(src_tokens, num_steps, src_vocab['<pad>'])
    # Unsqueeze adds another dimension that works as the the batch axis here
    enc_X = torch.unsqueeze(torch.tensor(src_tokens, dtype=torch.long, device=device), dim=0)
    enc_outputs = net.encoder(enc_X, enc_valid_len)
    dec_state = net.decoder.init_state(enc_outputs, enc_valid_len)
    # Add the batch axis to the decoder now
    dec_X = torch.unsqueeze(torch.tensor([tgt_vocab['<bos>']], dtype=torch.long, device=device), dim=0)
    output_seq, attention_weight_seq = [], []
    for _ in range(num_steps):
        Y = net.decoder(dec_X, dec_state)[0]
        # We use the token with the highest prediction likelihood as the input
        # of the decoder at the next time step
        dec_X = Y.argmax(dim=2)
        pred = dec_X.squeeze(dim=0).type(torch.int32).item()
        # Save attention weights
        if save_attention_weights:
            attention_weight_seq.append(net.decoder.attention_weights)
            # Once the end-of-sequence token is predicted, the generation of the output sequence is complete
        if pred == tgt_vocab['<eos>']:
                break
        output_seq.append(pred)
    if len(output_seq)<2:
        
        if len(output_seq)==1: 
            return ''.join(tgt_vocab.to_tokens(output_seq[0])), attention_weight_seq   
        else:
            
            return "No output!", attention_weight_seq
    else:
        return ' '.join(tgt_vocab.to_tokens(output_seq)), attention_weight_seq

In [27]:
x_test

6273    sooo happy amazon sells one grocery store neig...
5739    running vanilla would made using vanilla beans...
3238    husband love hot chocolate auto shipment every...
6201    buying coffee least years love smooth taste hi...
8325    wolfgang puck coffee chef reseerve columbian d...
                              ...                        
6128    tastes delisious works efectively drink rigth ...
9991    good product high quality majority people post...
7873    chips family favorite absolutely fabulous guac...
3717    hot chocolate cups work like coffe cups coffee...
8955    lamb jerky soft treats first product tried zuk...
Name: cleaned_text, Length: 944, dtype: object

In [28]:
sample = x_test[6273]
actual = y_test[6273]

pred_sum, _ = predict_seq2seq(net, sample, src_vocab, tgt_vocab, 10, device)

print("Actual Text: ", sample)
print("Actual Summary: ", actual)
print("Predicted Summary: ", pred_sum)

Actual Text:  sooo happy amazon sells one grocery store neighborhood closing delicious perfect fiesta night
Actual Summary:  amazingly delicious
Predicted Summary:  great food
